# Hypothesis tests from confidence intervals

In [52]:
import pandas as pd
from scipy.stats import norm

from common import *

## Import confidence interval tables

In [5]:
met_ci_table = pd.read_csv(tables_path / 'met_flux_ci.csv')
hcys_ci_table = pd.read_csv(tables_path / 'hcys_flux_ci.csv')

In [11]:
# merge into a single dataframe
ci_table = pd.concat(
    [
        met_ci_table.assign(condition='met').set_index(['condition', 'cell_type', 'reaction']).sort_index(),
        hcys_ci_table.assign(condition='hcys').set_index(['condition', 'cell_type', 'reaction']).sort_index(),
    ]
)

In [14]:
ci_table

net     std err  ci_half-width  \
condition cell_type reaction                                          
met       BJ-RAS    CYSTS       0.672216    0.111653       0.183652   
                    HCYS_OUT  132.374727    5.488244       9.027358   
                    METS      203.571238   61.183321     100.637607   
                    MET_IN    641.592467  124.277554     204.418385   
                    PROT_OUT  508.545524  124.156310     204.218957   
                    SAM_METH  336.618181   60.932934     100.225758   
          BJ-TERT   CYSTS       3.918462    0.734904       1.208810   
                    HCYS_OUT  149.877087   31.719494      52.173925   
                    METS       26.037927   37.608795      61.860963   
                    MET_IN    569.383462   93.623437     153.996850   
                    PROT_OUT  415.587913   88.104056     144.918275   
                    SAM_METH  179.833476   20.072712      33.016674   
hcys      BJ-RAS    CYSTS       2.113185    0.373083       0.613667   
                    HCYS_IN    66.493245   18.906584      31.098564   
                    METS       97.979346   22.090292      36.335297   
                    MET_OUT     3.645315    1.785672       2.937169   
                    PROT_OUT   60.734745   18.814839      30.947656   
                    SAM_METH   33.599286   11.052983      18.180540   
          BJ-TERT   CYSTS       3.002684    0.824304       1.355859   
                    HCYS_IN   276.552904   73.946363     121.630944   
                    METS      290.067417   74.165573     121.991512   
                    MET_OUT    15.435811   14.060181      23.126940   
                    PROT_OUT  258.114410   73.453783     120.820721   
                    SAM_METH   16.517197    5.444020       8.954616   

                                ci_lower    ci_upper  
condition cell_type reaction                          
met       BJ-RAS    CYSTS       0.488564    0.855869  
                    HCYS_OUT  123.347369  141.402085  
                    METS      102.933631  304.208846  
                    MET_IN    437.174082  846.010852  
                    PROT_OUT  304.326567  712.764481  
                    SAM_METH  236.392423  436.843939  
          BJ-TERT   CYSTS       2.709652    5.127272  
                    HCYS_OUT   97.703161  202.051012  
                    METS      -35.823036   87.898891  
                    MET_IN    415.386611  723.380312  
                    PROT_OUT  270.669637  560.506188  
                    SAM_METH  146.816802  212.850150  
hcys      BJ-RAS    CYSTS       1.499518    2.726852  
                    HCYS_IN    35.394681   97.591808  
                    METS       61.644049  134.314643  
                    MET_OUT     0.708146    6.582484  
                    PROT_OUT   29.787089   91.682400  
                    SAM_METH   15.418746   51.779826  
          BJ-TERT   CYSTS       1.646825    4.358543  
                    HCYS_IN   154.921960  398.183848  
                    METS      168.075905  412.058929  
                    MET_OUT    -7.691129   38.562750  
                    PROT_OUT  137.293689  378.935131  
                    SAM_METH    7.562581   25.471813

## Hypothesis tests

Assume net flux to be a normal distributed variable $X$ with true mean $\mu$. We have an observation $\bar{x}$ of the mean $\bar{X}$ with standard standard error $s = \sigma / n$ (assumed known). The variable $Z = (\bar{X} - \mu) / (\sigma/n)$ is then $N(0, 1)$. For testing the difference between two estimates, the statistic $(\bar{X}_1 - \bar{X}_2)/\sqrt(s_1^2 + s_2^2)$ is also $N(0, 1)$.

In [72]:
# compute a two-sided test p-value for a test table with conditions in rows
def normal_diff_test(test_table):
    means = []
    std_errs = []
    for index, row in test_table.iterrows():
        means.append(row['net'])
        std_errs.append(row['std err'])
    # two-sample z-statistic 
    z = (means[1] - means[0]) / np.sqrt(std_errs[0]**2 + std_errs[1]**2)
    # two-sided tail p-value
    return (1 - norm.cdf(abs(z))) * 2

Slice out values of interest from the CI table and perform tests

In [73]:
normal_diff_test(ci_table.xs(('BJ-TERT', 'METS'), level=('cell_type', 'reaction')))

0.0014978340293727577

In [74]:
normal_diff_test(ci_table.xs(('BJ-RAS', 'METS'), level=('cell_type', 'reaction')))

0.10453265110500909

In [75]:
normal_diff_test(ci_table.xs(('BJ-TERT', 'SAM_METH'), level=('cell_type', 'reaction')))

3.9968028886505635e-15

In [76]:
normal_diff_test(ci_table.xs(('BJ-RAS', 'SAM_METH'), level=('cell_type', 'reaction')))

9.924033963493173e-07